# Import Statements

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import OCR_pipeline as OCR
import TopicModeling as TM
import jsonpickle
import gensim

[nltk_data] Downloading package punkt to /Users/kimberly/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Cleaning And Saving Extracted Text

In [3]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="XXX",
  database="forestry"
)
mycursor = mydb.cursor()

def getExtractedTextsFromCountries(countries):
    sqlStatement = "SELECT P1.filename, P1.pid, P2.country, P1.text from PolicyPaperPages P1 " \
                     "INNER JOIN PolicyPapers P2 " \
                     "on P1.filename = P2.filename WHERE"
    
    for i in range(len(countries)):
        whereClause = " P2.country = '%s' " % countries[i]
        sqlStatement += whereClause
        if len(countries) != 1 and i < len(countries) - 1:
            sqlStatement += "or"

    sqlStatement += "GROUP BY filename, pid ORDER BY filename, pid"
    mycursor.execute(sqlStatement)

    myresult = mycursor.fetchall()
    return combinePages(myresult)

def combinePages(myresult):
    curr_filename = myresult[0][0]
    curr_pages = []
    file_texts = []
    for result in myresult:
        if result[0] != curr_filename:
            file_texts.append([curr_filename, curr_pages])
            curr_pages = []
            curr_filename = result[0]
        else:
            curr_pages.append(str(result[3]))
        
    file_texts.append([curr_filename, curr_pages])
    return file_texts

In [4]:
key_words = ['afforestation','agriculture','animal welfare','artificial regeneration','biodiversity',
             'biological resources','biome','board','clean','coconut','conservation','control','database',
             'enforcement','environment','farm','financing','forest','forest protection','funding','fundraising',
             'land ','land use','landholder','law','measuring','mobilization','monitor','natural resources',
             'oversight','plant breeders','plants','pollution','preservation','produce','protection','qualification',
             'quality','registry','regulation','reporting','reserve','resource','restriction','results-driven',
             'rural','safeguard','species' ,'support','sustainable','technical submission','threatened species',
             'variety','verification','wastewater','water','watershed','wild life']

In [5]:
spanish_key_words = ['repoblación forestal', 'agricultura', 'bienestar de los animales', 'regeneración artificial', 
                     'biodiversidad', 'recursos biologicos', 'bioma', 'tablero', 'limpiar', 'Coco', 'conservación',
                     'controlar', 'base de datos', 'aplicación', 'ambiente', 'granja', 'financiación', 'bosque', 
                     'protección forestal', 'fondos', 'recaudación de fondos', 'tierra', 'uso del suelo', 
                     'terrateniente', 'ley', 'medición', 'movilización', 'monitor', 'recursos naturales', 'vigilancia',
                     'fitomejoradores', 'plantas', 'contaminación', 'preservación', 'Produce', 'proteccion',
                     'calificación', 'calidad', 'registro', 'regulación', 'reportando', 'reserva', 'recurso', 
                     'restricción', 'impulsado por resultados', 'rural', 'salvaguardia', 'especies', 'apoyo', 
                     'sostenible', 'sumisión técnica', 'especies amenazadas', 'variedad', 'verificación', 
                     'aguas residuales', 'agua', 'cuenca', 'fauna silvestre']

In [6]:
#dictionary for texts in terms of sentences
import re
import gc
textDictionary = {}
countries = ['India']
policyPapers = getExtractedTextsFromCountries(countries)
language_keywords = {}
language_keywords['english'] = key_words
language_keywords['spanish'] = spanish_key_words
for paper in policyPapers:
    full_text = ' '.join(paper[1])
    cleanedText = TM.toSentences(full_text, 'english', language_keywords['english'])
    textDictionary[paper[0]] = cleanedText

# LDA

In [7]:
from pprint import pprint
lemmatized_sents = []

# Print the Keywords in the 5 topics
for key in textDictionary:
    cleanedText = textDictionary[key]
    lemmatized_sents.extend(cleanedText)
    
corpus, lda_model = TM.LDA(lemmatized_sents)
pprint(lda_model.print_topics(-1))
doc_lda = lda_model[corpus]
#Next Step: Store LDA data in Policy Paper Object & other central file

[(0,
  '0.036*"state" + 0.034*"authority" + 0.030*"section" + 0.019*"shall" + '
  '0.018*"sub" + 0.017*"fund" + 0.016*"government" + 0.015*"national" + '
  '0.012*"act" + 0.012*"rule"'),
 (1,
  '0.019*"rural" + 0.017*"climate" + 0.016*"water" + 0.016*"health" + '
  '0.014*"urban" + 0.014*"change" + 0.013*"population" + 0.013*"development" + '
  '0.013*"disease" + 0.013*"capacity"'),
 (2,
  '0.020*"water" + 0.018*"high" + 0.016*"crop" + 0.015*"area" + 0.014*"forest" '
  '+ 0.009*"promote" + 0.009*"rd" + 0.009*"use" + 0.008*"efficiency" + '
  '0.008*"bilateral"'),
 (3,
  '0.064*"forest" + 0.018*"area" + 0.012*"specie" + 0.012*"conservation" + '
  '0.012*"biodiversity" + 0.011*"state" + 0.010*"management" + 0.008*"soil" + '
  '0.008*"cover" + 0.007*"water"'),
 (4,
  '0.028*"forest" + 0.019*"state" + 0.019*"area" + 0.018*"environment" + '
  '0.017*"pradesh" + 0.017*"land" + 0.015*"andhra" + 0.012*"change" + '
  '0.012*"india" + 0.011*"climate"'),
 (5,
  '0.049*"market" + 0.035*"livestock" 

In [8]:
# Save LDA
# lda_model.save("lda_model")

In [9]:
# Load LDA
lda_saved = gensim.models.ldamodel.LdaModel.load("lda_model")

In [10]:
lda_saved.print_topics(-1)

[(0,
  '0.038*"livestock" + 0.031*"market" + 0.023*"product" + 0.020*"produce" + 0.019*"notify" + 0.018*"committee" + 0.016*"agricultural" + 0.012*"shall" + 0.011*"mean" + 0.010*"sale"'),
 (1,
  '0.021*"member" + 0.013*"india" + 0.012*"government" + 0.012*"organisation" + 0.012*"forest" + 0.011*"environment" + 0.011*"water" + 0.009*"epco" + 0.009*"skmccc" + 0.008*"director"'),
 (2,
  '0.040*"water" + 0.020*"area" + 0.017*"agriculture" + 0.015*"high" + 0.014*"resource" + 0.014*"irrigation" + 0.013*"ground" + 0.012*"groundwater" + 0.011*"use" + 0.011*"crop"'),
 (3,
  '0.020*"area" + 0.020*"forest" + 0.017*"person" + 0.016*"officer" + 0.015*"property" + 0.011*"land" + 0.011*"offence" + 0.009*"commit" + 0.008*"shall" + 0.008*"government"'),
 (4,
  '0.043*"water" + 0.015*"fish" + 0.011*"state" + 0.011*"practice" + 0.010*"increase" + 0.010*"climatic" + 0.010*"agro" + 0.010*"availability" + 0.009*"different" + 0.009*"seed"'),
 (5,
  '0.020*"change" + 0.018*"development" + 0.017*"climate" + 0.

In [11]:
output = lda_model.print_topics(-1)
test = output[0][1]
test.split("+")

['0.036*"state" ',
 ' 0.034*"authority" ',
 ' 0.030*"section" ',
 ' 0.019*"shall" ',
 ' 0.018*"sub" ',
 ' 0.017*"fund" ',
 ' 0.016*"government" ',
 ' 0.015*"national" ',
 ' 0.012*"act" ',
 ' 0.012*"rule"']

In [16]:
import operator

def top_three_topics(lda_model, text):
    preprocess = simple_preprocess(text)
    common_dictionary = Dictionary([text.split(" ")])
    bow = common_dictionary.doc2bow(preprocess)
    
    document_topics = lda_model.get_document_topics(bow)
    topics_ratio = dict(document_topics)
    
    top_three = []
    for i in range(0, 3):
        try: 
            largest = max(topics_ratio.items(), key=operator.itemgetter(1))
            top_three.append(largest)
            del topics_ratio[largest[0]]
        except: 
            return top_three
    
    return top_three

In [39]:
topic_map = {
    0: "Market",
    1: "Government",
    2: "Climate Change",
    3: "Forestry Policy",
    4: "Land/Water Management",
    5: "Organization",
    6: "States", 
    7: "Area", 
    8: "Crops",
    9: "Forest Conservation",
    10: "Legal",
    11: "Forest",
    12: "Project/Mission", 
    13: "Agriculture", 
    14: "Mining Regulation"
}

def insertPaperTopics(mycursor, filename, page_topics):
    sql_insert = "INSERT INTO PaperTopics " \
        "(filename, pid, topic) " \
        "VALUES (%s, %s, %s)"
    for pid, top_three in page_topics.items():
        for topic in top_three:
            val = (filename, pid, topic)
            mycursor.execute(sql_insert, val)
    mydb.commit()
    
def inDatabase(mycursor, filename):
    mycursor.execute("SELECT filename FROM PaperTopics WHERE filename='%s'" % filename)
    return mycursor.fetchall()

In [40]:
# Preparation cell for get_document topics. Preprocessing
from gensim.corpora.dictionary import Dictionary
from gensim.utils import simple_preprocess, simple_tokenize

for doc in policyPapers:
    page_topics = {}
    mycursor = mydb.cursor()
    if not inDatabase(mycursor, doc[0]):
        for i in range(len(doc[1])):
            topics = top_three_topics(lda_model, doc[1][i])
            labels = [topic_map[t[0]] for t in topics]
            page_topics[i+1] = labels
        insertPaperTopics(mycursor, doc[0], page_topics)
    mycursor.close()

In [ ]:
# Retrieve the three topics for this specific document
topics = []
for item in top_three:
    topics.append(output[item[0]])
topics

In [ ]:
my = lda_model.print_topics()[0][1]

In [ ]:
re.findall(r"[a-zA-Z]+",my)

## Extract sentences by Keywords

### TODO: Have a feature on the web app so that the user can extract text using keywords

In [ ]:
#Try with the first document
policy_0 = policyPapers[0][1]
sentences = TM.ReturnSentence(policy_0)

In [ ]:
#This can be used as a preview feature
for s in sentences:
    if 'land' in s:
        print(s)

## Use Key Words to classify documents

In [ ]:
key_words = ['afforestation','agriculture','animal welfare','artificial regeneration','biodiversity',
             'biological resources','biome','board','clean','coconut','conservation','control','database',
             'enforcement','environment','farm','financing','forest','forest protection','funding','fundraising',
             'land ','land use','landholder','law','measuring','mobilization','monitor','natural resources',
             'oversight','plant breeders','plants','pollution','preservation','produce','protection','qualification',
             'quality','registry','regulation','reporting','reserve','resource','restriction','results-driven',
             'rural','safeguard','species' ,'support','sustainable','technical submission','threatened species',
             'variety','verification','wastewater','water','watershed','wild life']

In [ ]:
'artificial regeneration' in cleanWhitespaces

# Visualizations

### pyLDAvis

`pip install pyldavis`

Code source: https://markroxor.github.io/gensim/static/notebooks/gensim_news_classification.html
<br>
The area of the circles represent the prevalance of the topic. The length of the bars on the right represent the membership of a term in a particular topic. 

In [ ]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
# p = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
# pyLDAvis.save_html(p, 'pyLDAvis.html')

### WordCloud

`pip install wordcloud`

Code source: https://medium.com/@rudyb2001/making-sense-of-the-news-using-machine-learning-e13ed5f96cf0
<br>
WordCloud of Top 10 words in each topic with the sizes of the words proportional to the weights.

In [ ]:
len(lda_model.show_topics(formatted=False))

In [ ]:
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
from wordcloud import WordCloud, STOPWORDS
cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]

cloud = WordCloud(background_color='white',
                  width=2500,
                  height=1800,
                  max_words=10,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0)

topics = lda_model.show_topics(formatted=False)

fig, axes = plt.subplots(2, 5, figsize=(15,10), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
    fig.add_subplot(ax)
    topic_words = dict(topics[i][1])
    cloud.generate_from_frequencies(topic_words, max_font_size=300)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
    plt.gca().axis('off')

plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
plt.show()

# Storage